In [ ]:
# Install czmodel and dependencies (only needed once)
#! pip install --upgrade pip
#! pip install czmodel

# install some other useful packages

#!pip install sklearn
#!pip install livelossplot --quiet

In [ ]:
# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Train Simple U-Net for segmentation and use it inside ZEN

This notebook illustrates a **simple** workflow of training an ANN with [TensorFlow 2](https://www.tensorflow.org/) using the keras API and exporting the trained model to the [czmodel format](https://pypi.org/project/czmodel/) to be ready for use within the [Intellesis](https://www.zeiss.com/microscopy/int/products/microscope-software/zen-intellesis-image-segmentation-by-deep-learning.html) software inside ZEN.

* **The trained model is for demo purposes and trained on a small dataset.**
* **Therefore, this notebook is meant to be understood as a guide for exporting trained models**
* **The notebook does not show how train a model correctly.**

In [ ]:
# imports to train a simple TF2 + Keras model for segmentation

import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

from traintools.model import build_model
from livelossplot import PlotLossesKerasTF

# Optional: suppress TF warnings
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
print(tf.version.GIT_VERSION, tf.__version__)
print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## Training Pipeline
This section describes a **simple** training procedure that creates a trained Keras model - in this case a U-Net.

* Therefore, the pipelin only represents one possible custom training procedure.
* Such procedure will vary from case to case and will contain more sophisticated ways to generate an optimized model

In [ ]:
# Define the parameters for loading the training data

# adjust to jour needs
base_directory = r"c:\Users\m1srh\OneDrive - Carl Zeiss AG\Smart_Microscopy_Workshop\datasets"

# place the original and label *.png images here
imgfolder = "nucleus_data/images/"
#imgfolder = "mouse_embryo/images"

# masks images have one channel (0=background and 1=object)
maskfolder = 'nucleus_data/label/'
#maskfolder = "mouse_embryo/label"

# construct the final path to the images and labels
IMAGES_FOLDER = os.path.join(base_directory, imgfolder)
MASKS_FOLDER = os.path.join(base_directory, maskfolder)

In [ ]:
# define the number of channels inside the image to be segmented
CHANNELS = 1

# define bit depth for images  and masks
BITDEPTH_IMAGES = tf.dtypes.uint16
BITDEPTH_MASKS = tf.dtypes.uint16

# the model contains 2 classes, eg. object and background
NUM_CLASSES = 2

# tilesize for training which depends on the GPU memory size etc.
SZTR = 512

# Model and Training Parameters
KERNEL_SIZE = 3
DROPOUT = 0.15
LEARNING_RATE = 1e-3
BATCHSIZE = 1

In [ ]:
# define very simple augumentation function - can be extented as needed
def random_crop_and_pad_image_and_label(image, label, size):
    """Randomly crops image together with labels.

    Args:
      image: A Tensor with shape [D_1, ..., D_K, N]
      label: A Tensor with shape [D_1, ..., D_K, M]
      size: A Tensor with shape [K] indicating the crop size.
    Returns:
      A tuple of (cropped_image, cropped_label).
    """
    
    combined = tf.concat([image, label], axis=-1)
    image_shape = tf.shape(image)
    combined_pad = tf.image.pad_to_bounding_box(combined, 0, 0,
                                                tf.maximum(size[0], image_shape[0]),
                                                tf.maximum(size[1], image_shape[1]))
    
    last_label_dim = tf.shape(label)[-1]
    last_image_dim = tf.shape(image)[-1]
    
    combined_crop = tf.image.random_crop(combined_pad,
                                         size=tf.concat([size, [last_label_dim + last_image_dim]],
                                                        axis=0))
    
    return (combined_crop[:, :, :last_image_dim], combined_crop[:, :, last_image_dim:])

# helper function to reorder labels incase of multiple classes
def reorder_labels(mask):
    for i, unique_value in enumerate(np.unique(mask)):
        mask[mask == unique_value] = i
    return mask

In [ ]:
# This part contains the logic to read pairs of images and label masks for training.

# get the sample images
sample_images = sorted([os.path.join(IMAGES_FOLDER, f) for f in os.listdir(IMAGES_FOLDER)
                        if os.path.isfile(os.path.join(IMAGES_FOLDER, f))])

# get the maks
sample_masks = sorted([os.path.join(MASKS_FOLDER, f) for f in os.listdir(MASKS_FOLDER)
                       if os.path.isfile(os.path.join(MASKS_FOLDER, f))])

# load images as numpy arrays
images_loaded = np.asarray([tf.image.decode_image(tf.io.read_file(sample_path),
                                                  dtype=BITDEPTH_IMAGES,
                                                  channels=CHANNELS).numpy()
                            for sample_path in sample_images]).astype(np.float32)

masks_loaded = [tf.image.decode_image(tf.io.read_file(sample_path), dtype=BITDEPTH_MASKS).numpy()[...,0] for sample_path in sample_masks]

# load masks and create one_hot encoded masks
masks_loaded = [tf.convert_to_tensor(reorder_labels(mask_loaded).astype(np.uint8)) for mask_loaded in masks_loaded]
masks_loaded = np.asarray([tf.one_hot(mask_loaded, depth=NUM_CLASSES).numpy() for mask_loaded in masks_loaded])

# Remark: For details see [tf.one_hot](https://www.tensorflow.org/api_docs/python/tf/one_hot)
# tf.one_hot creates X channels from X labels: 1 => [0.0, 1.0], 0 => [1.0, 0.0]`

In [ ]:
# split the datasets into training, test and validation sets
def split_datasets(images, shuffel=False, testsize=0.2):
    
    # split into train and test set
    train, test = train_test_split(images, shuffle=False, test_size=testsize)
    
    # split training set into final training set and validation set
    train, val = train_test_split(train, shuffle=False, test_size=testsize)
    
    return train, test, val

# do the splitting
images_train, images_test, images_val = split_datasets(images_loaded, testsize=0.2)
masks_train, masks_test, masks_val = split_datasets(masks_loaded, testsize=0.2) 

In [ ]:
# check if the images fit to the labels - always a good idea ... :-)

#pair2show = len(images_loaded)
pair2show = 3

for i in range(pair2show):
    
    # cut out a smaller image for visualization
    cropped_image, cropped_mask = random_crop_and_pad_image_and_label(image=images_loaded[i, ...],
                                                                      label=masks_loaded[i, ...],
                                                                      size=[SZTR, SZTR])
  
    # show the images next to the labels
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(cropped_image[...,0])
    ax2.imshow(cropped_mask[...,0])

In [ ]:
# create a tfdatasets from the images and masks
tfdataset = tf.data.Dataset.from_tensor_slices((images_loaded, masks_loaded))
tfdataset_train = tf.data.Dataset.from_tensor_slices((images_train, masks_train))
tfdataset_test = tf.data.Dataset.from_tensor_slices((images_test, masks_test))
tfdataset_val = tf.data.Dataset.from_tensor_slices((images_val, masks_val))

# apply the augmentation function to the tfdataset = just a random crop and create batch datasets 

# complete set
tfdataset = tfdataset.map(lambda image, label: random_crop_and_pad_image_and_label(image,
                                                                                   label,
                                                                                   size=[SZTR, SZTR])).batch(BATCHSIZE)

# train set
tfdataset_train = tfdataset_train.map(lambda image, label: random_crop_and_pad_image_and_label(image,
                                                                                               label,
                                                                                               size=[SZTR, SZTR])).batch(BATCHSIZE)
# test set
tfdataset_test = tfdataset_test.map(lambda image, label: random_crop_and_pad_image_and_label(image,
                                                                                             label,
                                                                                             size=[SZTR, SZTR])).batch(BATCHSIZE)
# validation set
tfdataset_val = tfdataset_val.map(lambda image, label: random_crop_and_pad_image_and_label(image,
                                                                                           label,
                                                                                           size=[SZTR, SZTR])).batch(BATCHSIZE)

In [ ]:
# generate th U-Net model
model = build_model(start_filters=16,
                    numchannels=CHANNELS,
                    numclasses=NUM_CLASSES,
                    learning_rate=LEARNING_RATE,
                    perimagestd=False,
                    kernelsize=KERNEL_SIZE,
                    dropout=DROPOUT
                   )

# sho model summary (optional)
#model.summary()

### Fit the model to the loaded data
This part fits the model to the loaded data and evaluates it on the training data. In this test example we do not care about an actual evaluation of the model using validation and test data sets.

In [ ]:
# train the U-Net model

# define number of training epochs
num_epochs = 10

# fit the model to the data using the "training" set and for validataion the "validation" set
model.fit(tfdataset_train,
          callbacks=[PlotLossesKerasTF()],
          validation_data=(tfdataset_val),
          epochs=num_epochs)

In [ ]:
# get the loss and accuary values
loss, accuracy = model.evaluate(tfdataset_test)

# show the final accuracy achieved on the "test" set which was never used for "anything" so far
print("The model achieves {}% accuracy on the test data.".format(accuracy * 100))

In [ ]:
# create an iterator for the dataset
dataset_it = iter(tfdataset_test)

In [ ]:
# get an image and a mask from the dataset
for img, mask in dataset_it:

    #run the prediction and get the masks using argmax
    pred = model.predict(img)[0]
    pred = np.argmax(pred, axis=-1)

    # scale the image and shwo the results - just re-execute the cell to display a different image
    img_scaled = img.numpy()[0] / img.numpy().max()
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_figwidth(16)

    # show images and set titles
    ax1.imshow(img_scaled[...,0])
    ax2.imshow(pred, cmap="gray", vmin=0, vmax=1)
    ax3.imshow(mask[0,...,0])

    ax1.set_title('Image')
    ax2.set_title('Prediction')
    ax3.set_title('Label Mask')

## Create a CZModel from the trained Keras model

In this section we export the trained model to the CZModel format using the czmodel library and some additional meta data all possible parameter choices are described in the [ANN model specification](https://pypi.org/project/czmodel/) that can be found on the PyPi packager for `czmodel`.

### Define Meta-Data

We first define the meta-data needed to run the model within the Intellesis infrastructure. The `czmodel` package offers a named tuple `ModelMetadata` that allows to either parse as JSON file as described or to directly specify the parameters as shown below.

### Create a Model Specification Object

The export functions provided by the `czmodel` package expect a `ModelSpec` tuple that features the Keras model to be exported and the corresponding model metadata.

Therefore, we wrap our model and the `model_metadata` instance into a `ModelSpec` object.

In [ ]:
# those functions are provided by the PyPi package called czmodel (by ZEISS)
from czmodel.util.preprocessing import add_preprocessing_layers
from czmodel.model_metadata import ModelMetadata, ModelSpec
from czmodel import convert_from_model_spec, convert_from_json_spec

# Define the model metadata

model_name = 'Simple_Nuclei_SegmentationModel'
model_classes = ["Background", "Nuclei"]

#model_name = 'Embryo_DIC_simple'
#model_classes = ["Background", "Embryo"]

pixtype = 'Gray16'
bordersize = 64

model_metadata = ModelMetadata.from_params(name=model_name, 
                                           color_handling='ConvertToMonochrome',
                                           pixel_type=pixtype,
                                           classes=model_classes,
                                           border_size=bordersize)


# Create a model specification object used for conversion
model_spec = ModelSpec(model=model, model_metadata=model_metadata)

### Perform model export into *.czmodel file format

The `czmodel` library offers two functions to perform the actual export. 

* `convert_from_json_spec` allows to provide a JSON file with all information to convert a model in SavedModel format on disk to a `.czmodel` file that can be loaded with ZEN.
* `convert_from_model_spec` expects a `ModelSpec` object, an output path and name and optionally target spatial dimensions for the expected input of the exported model. From this information it creates a `.czmodel` file containing the specified model.

```python
convert_from_model_spec(model_spec=model_spec, 
                        output_path=folder_to_store_czmodel, 
                        output_name=name_of_the_model, 
                        spatial_dims=spatial_dims)
```

In [ ]:
# Define dimensions - ZEN Intellesis requires fully defined spatial dimensions.
# This is the tile size used by the ZEN TilingStrategy to pass an image to the SegmentationService.

# Define target spatial dimensions of the model for inference.
spatial_dims = 1024, 1024

# package and save as CZMODEL - for inference the U-Net model will be exported as ONNX
convert_from_model_spec(model_spec=model_spec, 
                        output_path='./czmodel_output', 
                        output_name=model_name, 
                        spatial_dims=spatial_dims)

# In the example above there will be a ""./czmodel_output/simple_nuclei_segmodel.czmodel" file saved on disk.

## Remarks
The generated .czmodel file can be directly loaded into ZEN Intellesis to perform segmentation tasks with the trained model.

If there is already a trained model in SavedModel format present on disk, it can also be converted
by providing a meta-data JSON file as described in the [ANN Model Specification](https://pypi.org/project/czmodel/).

The following JSON document describes the same meta-data applied in the use case above:

```json
{
"BorderSize": 64,
"ColorHandling": "ConvertToMonochrome",
"PixelType": "Gray16",
"Classes": ["Background", "Nuclei"],
"ModelPath": "saved_tf2_model_output",
}
```

This information can be copied to a file e.g. in the current working directory `./model_spec.json`
that also contains the trained model in SavedModel format e.g. generated by the following line:

In [ ]:
# save the trained TF2.SavedModel as a folder structure
# The folder + the JSON file can be also used to import the model in ZEN

model.save('./saved_tf2_model_output_dims_unset/') # save as TF.SavedModel without dims set

# add extra layer infront to define dimensions and save as TF2.SavedModel
add_preprocessing_layers(model, layers=None, spatial_dims=spatial_dims).save('./saved_tf2_model_output/')

**Example of a model XML description**

<img src="mdpics/model_xml.png" >

To import the newly created model just use the **`Import`** function of the Intellesis Trainable Segmentation module in ZEN.

<img src="mdpics/zen_import_model1.png" >

Select the **`simple_nuclei_segmodel.czmodel`** file and press the **`Open`** button.

<img src="mdpics/zen_import_model2.png" >

Use the IP-function **`Segmentation`** to segment an image using the imported CZMODEL (containing the trained network).

<img src="mdpics/zen_import_model_IPseg.png">

To use the trained model to analyse an image there are two main options

1. directly create an Image Analysis Setting based on the model (no class hierarchy, but very simple)
2. assign the trained model to s specific class inside a customized image analysis setting (shown below)

The crucial step (when not using option 1) is the Select the correct **`Class Segmentation Method`** inside the Image Analysis Wizard.

<img src="mdpics/zen_import_model_IA1.png">

Use the **`Select Model`** function to assign the trained model and the actual **class** (from the trained model) of interest to assign the model / class to the respective object inside the image analysis setting.

<img src="mdpics/zen_import_model_IA2.png" >

Now the trained model will be used to segment the image. The built-in ZEN Tiling Client automatically  to chunk the image and deal with complex dimensions, like Use the **`Scenes`** etc.

Additional Post-Processing options, incl. a Minimum Confidence Threshold can be applied to further refine the results.

<img src="mdpics/zen_import_model_IA3.png" >

Finally, the model can be loaded into ZEN by using the **Import** function on the **JSON file**. 

If the model is supposed to be provided to other parties it is usually easier to exchange .czmodel files instead of SavedModel directories with corresponding JSON meta-data files.

The `czmodel` library also provides a `convert_from_json_spec` function that accepts the above mentioned JSON file and creates a CZModel:

In [ ]:
# This is an additional way how to create a CZMODEL from a saved TF2 model on disk + JSON file.
# The currently recommended way to to create the CZMODEL directly by using czmodel.convert_from_model_spec
# the path to the TF2.SavedModel folder is defined in the JSON shown above

convert_from_json_spec(model_spec_path='model_spec_dims_unset.json',
                       output_path='model_from_json',
                       output_name = 'simple_nuclei_segmodel_from_json',
                       spatial_dims=spatial_dims)

convert_from_json_spec(model_spec_path='model_spec.json',
                       output_path='model_from_json_dims_unset',
                       output_name = 'simple_nuclei_segmodel_from_json',
                       spatial_dims=None)

Use the commands below from a terminal to present the notebook as a slideshow.

`
jupyter nbconvert Train_UNet_Export_as_CZMODEL_final.ipynb --to slides --post serve 
    --SlidesExporter.reveal_theme=serif 
    --SlidesExporter.reveal_scroll=True 
    --SlidesExporter.reveal_transition=none
`

Or install the [RISE Extension](https://rise.readthedocs.io/en/stable/) to display a slideshow directly from within the notebook.

In [ ]:
%%cmd
jupyter nbconvert Train_UNet_Export_as_CZMODEL_final.ipynb --to slides --post serve --SlidesExporter.reveal_theme=serif --SlidesExporter.reveal_scroll=True --SlidesExporter.reveal_transition=none